In [ ]:
# Dataset Plant Disease Detection Plan Path

/path/to/dataset/
│
├── train/
│   ├── wortel/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   ├── wortel_disease1/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   ├── tomat/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   ├── tomat_disease1/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── ...
│
├── val/
│   ├── wortel/
│   ├── wortel_disease1/
│   ├── tomat/
│   ├── tomat_disease1/
│   └── ...
│
└── test/
    ├── wortel/
    ├── wortel_disease1/
    ├── tomat/
    ├── tomat_disease1/
    └── ...


80% - Train
10% - Validation
10% - Test

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import shutil
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    GlobalAveragePooling2D,
)
from keras.layers.experimental import preprocessing
from keras.utils import plot_model

In [ ]:
PATH_DATASET = r"/splitedDataset"
TRAIN_DIR = r"/splitedDataset/train_ds"
VAL_DIR = r"/splitedDataset/val_ds"
TEST_DIR = r"/splitedDataset/test_ds"

In [ ]:
CLASSES_PLANT = ["wortel", "tomat", "timun", "sawi"]
CLASSES_DISEASE = ["healthy", "leaf_blight", "powdery_mildew", "unknown"]

In [ ]:
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 50

In [ ]:
def create_data_pipeline(train_dir, val_dir, test_dir):
    data_augmentation = tf.keras.Sequential([
        layers.RandomRotation(0.2),
        layers.RandomTranslation(0.2, 0.2),
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
        layers.GaussianNoise(0.1),
    ])

    # Preprocessing layer
    preprocessing = tf.keras.Sequential([
        layers.Rescaling(1./255),
        layers.Normalization()
    ])

    train_ds = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        seed=42,
        batch_size=32,
        label_mode="categorical",
        image_size=(256, 256),
    )

    validation_ds = tf.keras.utils.image_dataset_from_directory(
        VAL_DIR,
        seed=42,
        batch_size=32,
        label_mode="categorical",
        image_size=(256, 256),
    )

    test_ds = tf.keras.utils.image_dataset_from_directory(
        TEST_DIR,
        seed=42,
        batch_size=32,
        label_mode="categorical",
        image_size=(256, 256),
    )

    # Apply augmentation only to training data
    train_ds = train_ds.map(
        lambda x, y: (data_augmentation(x, training=True), y),
        num_parallel_calls=tf.data.AUTOTUNE,
    )

    # Apply preprocessing to all datasets
    train_ds = train_ds.map(
        lambda x, y: (preprocessing(x), y), num_parallel_calls=tf.data.AUTOTUNE
    )
    val_ds = val_ds.map(
        lambda x, y: (preprocessing(x), y), num_parallel_calls=tf.data.AUTOTUNE
    )
    test_ds = test_ds.map(
        lambda x, y: (preprocessing(x), y), num_parallel_calls=tf.data.AUTOTUNE
    )

    # Prefetch for performance
    train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_ds = validation_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, validation_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = create_data_pipeline(
    train_dir=TRAIN_DIR, val_dir=VAL_DIR, test_dir=TEST_DIR
)

In [ ]:
def create_model(num_plants, num_diseases):
    base_model = keras.applications.EfficientNetV2B0(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
    )

    # Progressive learning rate unfreeze
    base_model.trainable = True
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    # Create model with attention mechanism
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs)

    # Add attention mechanism
    attention = layers.GlobalAveragePooling2D()(x)
    attention = layers.Dense(1024, activation="relu")(attention)
    attention = layers.Dense(x.shape[-1], activation="sigmoid")(attention)
    attention = layers.Reshape((1, 1, x.shape[-1]))(attention)
    x = layers.Multiply()([x, attention])

    # Global pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Dropout for regularization
    x = layers.Dropout(0.5)(x)

    # Plant type branch
    plant_branch = layers.Dense(512, activation="relu")(x)
    plant_branch = layers.BatchNormalization()(plant_branch)
    plant_branch = layers.Dropout(0.3)(plant_branch)
    plant_output = layers.Dense(num_plants, activation="softmax", name="plant_type")(plant_branch)

    # Disease type branch
    disease_branch = layers.Dense(512, activation="relu")(x)
    disease_branch = layers.BatchNormalization()(disease_branch)
    disease_branch = layers.Dropout(0.3)(disease_branch)
    disease_output = layers.Dense(num_diseases, activation="softmax", name="disease_type")(disease_branch)

    # Create model
    model = keras.Model(inputs=inputs, outputs=[plant_output, disease_output])

    # Compile with weighted losses and learning rate scheduler
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps, decay_rate
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    model.compile(
        optimizer=optimizer,
        loss={
            "plant_type": "categorical_crossentropy",
            "disease_type": "categorical_crossentropy"
        },
        loss_weights={
            "plant_type": 1.0,
            "disease_type": 1.0
        },
        metrics={
            "plant_type": ["accuracy", tf.keras.metrics.AUC()],
            "disease_type": ["accuracy", tf.keras.metrics.AUC()]
        }
    )

    return model

In [ ]:
print("Creating model...")
model = create_model(num_plants=len(CLASSES_PLANT), num_diseases=len(CLASSES_DISEASE))

In [ ]:
def train_model(model, train_ds, val_ds, epochs=EPOCHS):
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6
        ),
        tf.keras.callbacks.ModelCheckpoint(
            "best_model.h5", monitor="val_loss", save_best_only=True
        ),
    ]

    history = model.fit(
        train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks
    )

    return history, model

In [ ]:
print("Starting training...")
history, trained_model = train_model(
    model=model,
    train_ds=train_ds,
    val_ds=val_ds,
    epochs=50,  # Sesuaikan dengan kebutuhan
)

In [ ]:
# Evaluate the Model
print("Evaluating model...")
test_results = trained_model.evaluate(test_ds)
print("\nTest Results:")
print(f"Plant Type Loss: {test_results[1]:.4f}")
print(f"Plant Type Accuracy: {test_results[3]:.4f}")
print(f"Plant Type AUC: {test_results[4]:.4f}")
print(f"Disease Type Loss: {test_results[2]:.4f}")
print(f"Disease Type Accuracy: {test_results[5]:.4f}")
print(f"Disease Type AUC: {test_results[6]:.4f}")

In [ ]:
# Visualize Training History
import matplotlib.pyplot as plt


def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot akurasi
    ax1.plot(history.history["plant_type_accuracy"], label="Plant Training Accuracy")
    ax1.plot(
        history.history["val_plant_type_accuracy"], label="Plant Validation Accuracy"
    )
    ax1.plot(
        history.history["disease_type_accuracy"], label="Disease Training Accuracy"
    )
    ax1.plot(
        history.history["val_disease_type_accuracy"],
        label="Disease Validation Accuracy",
    )
    ax1.set_title("Model Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Accuracy")
    ax1.legend()

    # Plot loss
    ax2.plot(history.history["plant_type_loss"], label="Plant Training Loss")
    ax2.plot(history.history["val_plant_type_loss"], label="Plant Validation Loss")
    ax2.plot(history.history["disease_type_loss"], label="Disease Training Loss")
    ax2.plot(history.history["val_disease_type_loss"], label="Disease Validation Loss")
    ax2.set_title("Model Loss")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Loss")
    ax2.legend()

    plt.tight_layout()
    plt.show()


plot_training_history(history)

In [ ]:
# Evaluate the Model
eval_results = model.evaluate(validation_data)
print(f"Plant Type Accuracy: {eval_results[1]:.2f}")
print(f"Disease Type Accuracy: {eval_results[2]:.2f}")

In [ ]:
# Save Model
print("Saving model...")
trained_model.save("plant_disease_model_final.h5")
print("Model saved as 'plant_disease_model_final.h5'")

In [ ]:
def predict_image(model, image_path):
    # Load and preprocess image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = img_array / 255.0

    # Predict
    plant_pred, disease_pred = model.predict(img_array)

    # Choose Highest Confidence
    plant_class = CLASSES_PLANT[tf.argmax(plant_pred[0])]
    disease_class = CLASSES_DISEASE[tf.argmax(disease_pred[0])]

    return {
        "plant": plant_class,
        "plant_confidence": float(tf.reduce_max(plant_pred[0])),
        "disease": disease_class,
        "disease_confidence": float(tf.reduce_max(disease_pred[0])),
    }

In [ ]:
result = predict_image(trained_model, 'path/to/image.jpg')
print(result)

In [ ]:
from sklearn.metrics import f1_score


def f1_score(y_true, y_pred, average="macro"):
    return f1_score(y_true, y_pred, average=average)


# Calculate F1 score
y_true = np.argmax(test_ds.labels, axis=1)
y_pred = np.argmax(model.predict(test_ds), axis=1)
f1_score = f1_score(y_true, y_pred, average="macro")

# Print the F1 score
print(f"F1 score: {f1_score}")